In [ ]:
%matplotlib widget

import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import ipywidgets as widgets
from functools import partial
import networkx as nx
import json
import numpy as np

from ophys_etl.modules.segmentation.graph_utils import plotting as guplot
from ophys_etl.modules.decrosstalk.ophys_plane import OphysROI, OphysMovie
from ophys_etl.modules.decrosstalk.ophys_plane import find_overlapping_roi_pairs
from ophys_etl.modules.segmentation.qc_utils.roi_utils import ROIExaminer
from ophys_etl.modules.segmentation.qc_utils.roi_utils import add_roi_boundaries_to_img, add_labels_to_axes
from ophys_etl.modules.segmentation.qc_utils.roi_utils import roi_thumbnail

In [ ]:
basedir = Path("/allen/programs/braintv/workgroups/nc-ophys/danielk/deepinterpolation/experiments")
experiment_ids = [785569470, 1048483611, 1048483613, 1048483616, 785569447,
                  788422859, 795901850, 788422825, 795897800, 850517348,
                  951980473, 951980484, 795901895, 806862946, 803965468,
                  806928824]
metadata_files = [basedir / f"ophys_experiment_{i}" / "metadata.csv" for i in experiment_ids]

### Available Experiments

In [ ]:
frames = pd.concat([pd.read_csv(file, header=None).set_index(0).T for file in metadata_files]).set_index("ophys_experiment_id")
frames["depth"] = frames["depth"].astype(int)
frames = frames.sort_values(by=["code", "rig", "depth"]).reset_index()
display(frames)

In [ ]:
df_per_row = [pd.DataFrame(i[1]).T for i in frames.iterrows()]
col_widths = [max([len(f"{i}") for i in frames[c].values]) for c in frames.columns]
options = ["".join([f"{i:{w + 4}}" for i, w in zip(k.values[0], col_widths)]) for k in df_per_row]
eids = [int(i["ophys_experiment_id"].values[0]) for i in df_per_row]
values = [basedir / f"ophys_experiment_{i}" for i in eids]
experiment_selector = widgets.Dropdown(
    options=[(o, v) for o, v in zip(options, values)],
    layout={'width': 'max-content'}, # If the items' names are long
    description='Select experiment:',
    disabled=False
)
display(experiment_selector)

In [ ]:
def get_max_ave_projections(result_dir: Path):
    nmax = result_dir / "noised_maxp.png"
    navg = result_dir / "noised_avgp.png"
    dnmax = result_dir / "denoised_maxp.png"
    dnavg = result_dir / "denoised_avgp.png"

    nmaxim = np.array(Image.open(nmax))
    navgim = np.array(Image.open(navg))
    dnmaxim = np.array(Image.open(dnmax))
    dnavgim = np.array(Image.open(dnavg))

    return nmaxim, dnmaxim, navgim, dnavgim

In [ ]:
def new_background_selector(nrows, ncols):
    available = []
    for ext in ["png"]:
        available.extend(
            sorted(list((experiment_selector.value / "backgrounds").rglob(f"*.{ext}"))))
    background_selector = [
        widgets.Dropdown(
        options = [(None, None)] + [(p.name, p) for p in available],
        description = f"({i}, {j})")
    for i in range(nrows)
    for j in range(ncols)]
    return background_selector

def new_foreground_selector(nrows, ncols):
    files = sorted(list((experiment_selector.value / "rois").rglob("*_rois.json")))
    foreground_selector = [
        widgets.Dropdown(
        options = [(None, None)] + [(f.name, f) for f in files],
        description = f"({i}, {j})")
    for i in range(nrows)
    for j in range(ncols)
    ]
    return foreground_selector

def new_plot_update_buttons(nrows, ncols):
    buttons = [
        widgets.Button(description="Update")
    for i in range(nrows)
    for j in range(ncols)
    ]
    return buttons

In [ ]:
def update_plot(widget, axes, background_widget, foreground_widget):
    print(widget, axes, background_widget, foreground_widget)
    background_path = background_widget.value
    if background_path is None:
        im = np.ones((512, 512, 3), dtype="uint8") * 255
    else:
        im = plt.imread(background_path)
        if im.ndim == 2:
            im = np.dstack([im, im, im])
    
    foreground_path = foreground_widget.value
    if foreground_path is not None:
        with open(foreground_path, "r") as f:
            rois = json.load(f)
        im = add_roi_boundaries_to_img(im, rois)
    axes.cla()
    axes.imshow(im)
    axes.set_title(f"{background_path.name}\n{foreground_path.name}")

@widgets.interact(nrows=[1, 2, 3], ncols=[1, 2, 3])
def update(nrows=1, ncols=1):
    # erase old figure
    fig = plt.figure(1)
    plt.close(fig)
    
    # make new figure
    fig, axes = plt.subplots(nrows, ncols, clear=True, sharex=True, sharey=True, num=1, squeeze=False)
    plt.show()
    
    # make selectors for each axis and attach to callbacks
    backgrounds = new_background_selector(nrows, ncols)
    foregrounds = new_foreground_selector(nrows, ncols)
    partials = []
    for ax, bgw, fgw in zip(axes.flat, backgrounds, foregrounds):
        partials.append(partial(update_plot,
                                axes=ax,
                                foreground_widget=fgw,
                                background_widget=bgw))
    update_buttons = new_plot_update_buttons(nrows, ncols)
    for partial_fun, button in zip(partials, update_buttons):
        button.on_click(partial_fun)
    
    # group the selectors and display
    background_box = widgets.VBox([widgets.Label("backgrounds")] + backgrounds)
    foreground_box = widgets.VBox([widgets.Label("foregrounds")] + foregrounds)
    button_box = widgets.VBox([widgets.Label("update buttons")] + update_buttons)
    selector_box = widgets.HBox([background_box, foreground_box, button_box])
    display(selector_box)

In [ ]:
movie_widget_list = [
    widgets.Checkbox(
        value=True,
        description=f.name,
        layout={'width': 'max-content'}
    )
    for f in sorted(list((experiment_selector.value / "videos").rglob("*.h5")))
]
rois_lists = sorted(list((experiment_selector.value / "rois").rglob("*_rois.json")))
rois_dict = dict()
for r in rois_lists:
    with open(r, "r") as f:
        rois_dict[r.name] = json.load(f)
movie_list = widgets.VBox(movie_widget_list)
roi_drops = [
    widgets.Dropdown(
        options=np.sort([-1] + [i["id"] for i in v]),
        description=k,
        layout={'width': 'max-content'},
        style={'description_width': 'initial'}
    )
    for k, v in rois_dict.items()]
roi_list = widgets.VBox(roi_drops)
display(widgets.HBox(
    [widgets.VBox([widgets.HTML(value="<b>available movies</b>"),
                   movie_list]),
     widgets.VBox([widgets.HTML(value="<b>available ROIs</b>"),
                   roi_list])],
    layout={'width': 'max-content'}))

In [ ]:
def plot_callback(widg):
    rois_lookup = dict()
    for roi_select in roi_drops:
        print(roi_select.value)
        if roi_select.value != -1:
            rois_lookup[roi_select.description] = int(roi_select.value)
    rois = []
    for k, v in list(rois_lookup.items()):
        with open(experiment_selector.value / "rois" / k, "r") as f:
            j = json.load(f)
        for i in j:
            if i["id"] == v:
                rois_lookup[k] = i
    print(rois_lookup)
button = widgets.Button(
    description="plot traces",
)
button.on_click(plot_callback)
display(button)